## Random Forest Regressor Implementation from Scratch

In [55]:
import numpy as np
import pandas as pd
import operator

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae , mean_squared_error as mse, root_mean_squared_error as rmse, r2_score as r2

In [56]:
ops = {
    '<=' : operator.le,
    '>' : operator.gt
}

In [57]:
diabetes_sklearn = load_diabetes(as_frame=True)
print(diabetes_sklearn.DESCR)

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

:Number of Instances: 442

:Number of Attributes: First 10 columns are numeric predictive values

:Target: Column 11 is a quantitative measure of disease progression one year after baseline

:Attribute Information:
    - age     age in years
    - sex
    - bmi     body mass index
    - bp      average blood pressure
    - s1      tc, total serum cholesterol
    - s2      ldl, low-density lipoproteins
    - s3      hdl, high-density lipoproteins
    - s4      tch, total cholesterol / HDL
    - s5      ltg, possibly log of serum triglycerides level
    - s6      glu, blood sugar level

Note: Each of these 10 feature variables have bee

In [58]:
diabetes: pd.DataFrame= diabetes_sklearn.frame
diabetes.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


In [59]:
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     442 non-null    float64
 1   sex     442 non-null    float64
 2   bmi     442 non-null    float64
 3   bp      442 non-null    float64
 4   s1      442 non-null    float64
 5   s2      442 non-null    float64
 6   s3      442 non-null    float64
 7   s4      442 non-null    float64
 8   s5      442 non-null    float64
 9   s6      442 non-null    float64
 10  target  442 non-null    float64
dtypes: float64(11)
memory usage: 38.1 KB


In [60]:
diabetes.describe()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
count,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,442.000000
mean,-2.511817e-19,1.230790e-17,-2.245564e-16,-4.797570e-17,-1.381499e-17,3.918434e-17,-5.777179e-18,-9.042540e-18,9.293722e-17,1.130318e-17,152.133484
std,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,77.093005
min,-1.072256e-01,-4.464164e-02,-9.027530e-02,-1.123988e-01,-1.267807e-01,-1.156131e-01,-1.023071e-01,-7.639450e-02,-1.260971e-01,-1.377672e-01,25.000000
25%,-3.729927e-02,-4.464164e-02,-3.422907e-02,-3.665608e-02,-3.424784e-02,-3.035840e-02,-3.511716e-02,-3.949338e-02,-3.324559e-02,-3.317903e-02,87.000000
50%,5.383060e-03,-4.464164e-02,-7.283766e-03,-5.670422e-03,-4.320866e-03,-3.819065e-03,-6.584468e-03,-2.592262e-03,-1.947171e-03,-1.077698e-03,140.500000
75%,3.807591e-02,5.068012e-02,3.124802e-02,3.564379e-02,2.835801e-02,2.984439e-02,2.931150e-02,3.430886e-02,3.243232e-02,2.791705e-02,211.500000
max,1.107267e-01,5.068012e-02,1.705552e-01,1.320436e-01,1.539137e-01,1.987880e-01,1.811791e-01,1.852344e-01,1.335973e-01,1.356118e-01,346.000000


In [61]:
X: pd.DataFrame = diabetes.drop("target", axis=1)
y: pd.Series = diabetes["target"]

In [62]:
X: np.ndarray = X.to_numpy()
y: np.ndarray = y.to_numpy()

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [64]:
class MSE:    
    def mse(self, y_true: np.ndarray, y_pred) -> float:
        mse = np.sum((y_true - y_pred)**2) / y_true.shape[0]
        return mse
    
    def loss(self, left_samples: int, left_mse: float, right_samples: int, right_mse: float) -> float:
        loss = (left_samples*left_mse + right_samples*right_mse) / (left_samples + right_samples)
        return loss

In [65]:
class Node:
    def __init__(self, mse: float = None, samples: int = None, value: float = None, feature: int = None, threshold: float = None):
        self.feature = feature
        self.threshold = threshold
        self.mse = mse
        self.samples = samples
        self.value = value
               
        self.left: Node = None
        self.right: Node = None
        
    def __repr__(self) -> str:
        if self.feature is None:
            return f'Leaf: samples={self.samples}, value = {self.value}'
        return f'X[{self.feature}] <= {self.threshold:.3f}, mse={self.mse:.3f}, samples={self.samples}, value = {self.value}'

In [66]:
class DecisionTree:
    def __init__(self, max_depth: int = None, max_features: int = None):
        self.root: Node = None
        self.mse = MSE()
        self.max_depth = max_depth
        self.max_features = max_features
        
    def create_best_split(self, parent_node: Node, X: np.ndarray, y: np.ndarray):
        best_mse = parent_node.mse
        n_samples, n_features = X.shape
        features = np.random.choice(n_features, self.max_features, replace=False)
        
        for n in features:
            sorted_ids = np.argsort(X[:, n])
            thresholds = X[:, n][sorted_ids]
            labels = y[sorted_ids]
            cum_sum = np.cumsum(labels)
            cum_sq_sum = np.cumsum(labels**2)
            
            for m in range(0, len(thresholds)-1):
                if thresholds[m+1] == thresholds[m]:
                    continue
                
                left_samples = m + 1
                left_mean = cum_sum[m] / left_samples
                left_mse = (cum_sq_sum[m] / left_samples) - left_mean**2
                
                right_samples = n_samples - left_samples 
                right_mean = (cum_sum[-1] - cum_sum[m]) / right_samples
                right_mse = ((cum_sq_sum[-1] - cum_sq_sum[m]) / right_samples) - right_mean**2
                
                new_mse = self.mse.loss(left_samples, left_mse, right_samples, right_mse)
                
                if new_mse < best_mse:
                    best_mse = new_mse
                    parent_node.feature = n
                    parent_node.threshold = (thresholds[m] + thresholds[m+1]) / 2
                    parent_node.left = Node(left_mse, left_samples, left_mean)
                    parent_node.right = Node(right_mse, right_samples, right_mean)
                    
    def create_nodes(self, parent_node: Node, X: np.ndarray, y: np.ndarray, max_depth: int):
        if max_depth == 0 or len(np.unique(y)) == 1 or parent_node.samples <= 1:
            return
        
        self.create_best_split(parent_node, X, y)

        if parent_node.feature is None or parent_node.threshold is None:
            return
                 
        split_mask = X[:, parent_node.feature] <= parent_node.threshold
        X_left, y_left = X[split_mask], y[split_mask]
        X_right, y_right = X[~split_mask], y[~split_mask]   
        
        self.create_nodes(parent_node.left, X_left, y_left, None if max_depth is None else max_depth - 1)
        self.create_nodes(parent_node.right, X_right, y_right, None if max_depth is None else max_depth - 1)
                  
    def build_tree(self, X: np.ndarray, y: np.ndarray):
        samples = len(y)
        value = np.mean(y)
        mse = self.mse.mse(y, value)
        self.root = Node(mse, samples, value)
        self.create_nodes(self.root, X, y, self.max_depth)
    
    def check_value(self, x: np.ndarray) -> float:
        current_node = self.root

        while current_node.feature is not None:
            if x[current_node.feature] <= current_node.threshold:
                current_node = current_node.left
            else:
                current_node = current_node.right
   
        return current_node.value
    
    def pre_order_traversal(self, node: Node, depth: int):
        if node is None:
            return
        indent = "  " * depth
        print(indent, node)
        self.pre_order_traversal(node.left, depth + 1)
        self.pre_order_traversal(node.right, depth + 1)    
        
    def print_tree(self):
        self.pre_order_traversal(self.root, 0) 

In [67]:
class DecisionTreeRegressor(DecisionTree):
    def __init__(self, max_depth: int = None, max_features = None):
        super().__init__(max_features=max_features)
        self.max_depth = max_depth
        
    def _resolve_max_features(self, X: np.ndarray) -> int:
        n_features = X.shape[1]

        if self.max_features is None:
            return n_features
        elif isinstance(self.max_features, str):
            if self.max_features == "sqrt":
                return max(1, int(np.sqrt(n_features)))
            elif self.max_features == "log2":
                return max(1, int(np.log2(n_features)))
            else:
                raise ValueError(f"Unknown max_features string: {self.max_features}")
        elif isinstance(self.max_features, int):
            return max(1, min(self.max_features, n_features))
        elif isinstance(self.max_features, float):
            if not (0.0 < self.max_features <= 1.0):
                raise ValueError("If max_features is float, it must be in [0,1].")
            return max(1, int(self.max_features * n_features))
        else:
            raise TypeError("max_features must be None, int, float, or str {'sqrt', 'log2'}.")
           
    def fit(self, X: np.ndarray, y: np.ndarray):
        self.max_features = self._resolve_max_features(X)
        self.build_tree(X, y)
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        y_pred = np.array([self.check_value(x) for x in X])
        return y_pred

In [68]:
class RandomForestRegressor:
    def __init__(self, n_estimators: int = 100, max_features = "sqrt", bootstrap: bool = True):
        self.n_estimators = n_estimators
        self._estimators: list[DecisionTreeRegressor] = []
        self.max_features = max_features
        self.bootstrap = bootstrap
    
    def bootstrap_sample(self, X: np.ndarray, y: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
        n_samples = X.shape[0]
        bstr_ids = np.random.choice(n_samples, n_samples)
        
        return X[bstr_ids], y[bstr_ids]
    
    def fit(self, X: np.ndarray, y: np.ndarray):      
        for _ in range(self.n_estimators):
            if self.bootstrap:    
                X_sample, y_sample = self.bootstrap_sample(X, y)
            else: 
                X_sample, y_sample = X, y
            d_tree_model = DecisionTreeRegressor(max_features=self.max_features)
            d_tree_model.fit(X_sample, y_sample)
            self._estimators.append(d_tree_model)
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        y_pred = np.mean([est.predict(X) for est in self._estimators], axis=0)

        return y_pred

In [69]:
model = RandomForestRegressor()

In [70]:
model.fit(X_train, y_train)

In [71]:
y_pred = model.predict(X_test)

In [72]:
print('MAE:', mae(y_test, y_pred))
print('MSE:', mse(y_test, y_pred))
print('RMSE:', rmse(y_test, y_pred))
print('R2:', r2(y_test, y_pred))

MAE: 46.8096936936937
MSE: 3242.605322506506
RMSE: 56.943878709713005
R2: 0.5019488752588179
